In [1]:
from __future__ import print_function
import datetime
print(datetime.datetime.now())
import http.server
import urllib.parse
from urllib.parse import urlparse
import matplotlib
import datetime
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import urllib3
import urllib
from urllib.request import urlretrieve
import time
import pyodbc
import sys
from shutil import copyfile
import concurrent.futures
import os
import gc

print("OK")


2020-06-30 03:23:40.752846
OK


In [2]:
DBSTRING = DBSTRING = "DRIVER={SQL Server};Server=LAPTOP-Q26NOFQQ\ATOMLOCALDB;Trusted_Connection=yes;Persist Security Info=false;Integrated Security=sspi;connection timeout=60;database=AtomLocalDB;"
print(DBSTRING)
connection = pyodbc.connect(DBSTRING)


DRIVER={SQL Server};Server=LAPTOP-Q26NOFQQ\ATOMLOCALDB;Trusted_Connection=yes;Persist Security Info=false;Integrated Security=sspi;connection timeout=60;database=AtomLocalDB;


In [24]:
processDate = "2020-03-13"

histData_sql = """select * from HistoricalData with(nolock) where Instrument like 'BANKNIFTY%MAR%' 
            and CONVERT(Date, DateInfo) = Convert(Date, '""" + processDate + """') 
            and  frequency in ('3minute') 
            order by DateInfo asc"""

dfHist = pd.read_sql(histData_sql, connection)
print(dfHist.columns)

Index(['Market', 'Instrument', 'DateInfo', 'OpenPx', 'HighPx', 'LowPx',
       'closePx', 'Volume', 'Frequency', 'CandleDate', 'PMS5', 'PMP5', 'PME5',
       'PMS10', 'PMP10', 'PME10', 'CandleId', 'PMS20', 'PMP20', 'PME20',
       'PMS50', 'PMP50', 'PME50', 'MACD', 'MACDSignal', 'MACDHisto', 'EMA26',
       'EMA12', 'EMAU', 'EMAD', 'RSI', 'PMS200', 'Source', 'ServerDate'],
      dtype='object')


In [12]:
observations_sql = """select * from Atom_AI_Candle_Observations_For_RL where InstrumentKey = 'BANKNIFTY' 
            and CONVERT(Date, TradeDate) = Convert(Date, '""" + processDate + """') 
            and  Eventfrequency in ('3minute') 
            order by TimeAtEventOpen asc"""

dfObsv = pd.read_sql(observations_sql, connection)
print(dfObsv.columns)

Index(['Market', 'InstrumentKey', 'Instrument', 'TradeDate', 'MZ_GroupCode',
       'TimeAtEventOpen', 'TimeAtEventClose', 'EventType', 'EventDetail',
       'EventsBefore',
       ...
       'Relation_MomentumFast', 'Relation_MomentumSlow', 'Value_DayHLRange',
       'Relation_CandlevsCH3', 'Relation_CandlevsCL3',
       'LIST_PivotsInProximity', 'Candle_HighPx', 'Candle_LowPx',
       'Candle_ClosePx', 'Candle_Volume'],
      dtype='object', length=133)


In [14]:
tz_sql = """select * from Atom_Trend_Zone_Record_Info where InstrumentKey = 'BANKNIFTY' 
            and MainDate = Convert(Date, '""" + processDate + """') 
            order by TimeAtEventOpen asc"""

dfTZ = pd.read_sql(tz_sql, connection)
print(dfTZ.columns)

Index(['InstrumentKey', 'MainDate', 'MZ_GroupCode', 'TimeAtEventOpen',
       'candle_PMA_Trend', 'candle_PricePMA_Trend', 'candle_Final_Trend',
       'candle15_PMA_Trend', 'candle15_PricePMA_Trend', 'candle15_Final_Trend',
       ...
       'liveLast60MinuteCandlePMA200', 'liveYesterdayDayCandlePMA20',
       'liveYesterdayDayCandlePMA50', 'liveYesterdayDayCandlePMA200',
       'lastTMinus1120MinuteAnchor_OpenPx',
       'lastTMinus1120MinuteAnchor_HighPx', 'lastTMinus1120MinuteAnchor_LowPx',
       'lastTMinus1120MinuteAnchor_ClosePx', 'label1', 'label2'],
      dtype='object', length=208)


In [15]:
beh_sql = """select * from Atom_AI_BehaviorMatrix_SpecialStatus where InstrumentKey = 'BANKNIFTY' 
            and MainDate = Convert(Date, '""" + processDate + """') 
            order by TradeDate asc"""

dfBehave = pd.read_sql(beh_sql, connection)
print(dfBehave.columns)


Index(['InstrumentKey', 'MainDate', 'TradeDate', 'TradeTimeOnly',
       'MZ_GroupCode', 'MainCode', 'MainCodeKey', 'LastStatus',
       'LastStatusChangeCount', 'NewStatusChange', 'NewStatusChangeCount',
       'TotalStatusChangeCount', 'TPOChangeLog', 'LastStatusLevel',
       'NewStatusLevel', 'StatusDistance_Max', 'StatusDistance_Min',
       'StatusDistance', 'StatusDistance_Effort', 'StatusDistanceVSEffort',
       'StatusDuration', 'StatusDistancePercent'],
      dtype='object')


In [16]:
fpmap_sql = """select * from Atom_FlightPlanZone_T0_StaticMap where InstrumentKey = 'BANKNIFTY' 
            and MainDate = Convert(Date, '""" + processDate + """') 
            """

dfFpMap = pd.read_sql(fpmap_sql, connection)
print(dfFpMap.columns)


Index(['InstrumentKey', 'MainDate', 'ZoneDescription', 'ZoneColor',
       'ZoneDescribe', 'ZoneTopBand', 'ZoneLowBand', 'ZoneWidth',
       'ZoneSentimentWeight', 'ZoneSuperPivotCount', 'ZoneTotalPivotcount',
       'ZoneTopBandDistanceFromAnchor', 'ZoneLowBandDistanceFromAnchor',
       'ZoneSuperPivotCountRoadToZone', 'ZoneTotalPivotcountRoadToZone',
       'AnchorValue'],
      dtype='object')


In [29]:
dfHist.head

<bound method NDFrame.head of     Market         Instrument            DateInfo   OpenPx    HighPx  \
0      NFO  BANKNIFTY20MARFUT 2020-03-13 09:15:00  22614.1  22678.00   
1      NFO  BANKNIFTY20MARFUT 2020-03-13 09:18:00  22050.0  22188.95   
2      NFO  BANKNIFTY20MARFUT 2020-03-13 09:21:00  21407.4  21417.95   
3      NFO  BANKNIFTY20MARFUT 2020-03-13 10:21:00  21998.0  23100.00   
4      NFO  BANKNIFTY20MARFUT 2020-03-13 10:24:00  22800.0  22800.00   
..     ...                ...                 ...      ...       ...   
106    NFO  BANKNIFTY20MARFUT 2020-03-13 15:30:00  25250.0  25250.00   
107    NFO  BANKNIFTY20MARFUT 2020-03-13 16:18:00  25250.0  25250.00   
108    NFO  BANKNIFTY20MARFUT 2020-03-13 17:03:00  25250.0  25250.00   
109    NFO  BANKNIFTY20MARFUT 2020-03-13 17:06:00  25250.0  25250.00   
110    NFO  BANKNIFTY20MARFUT 2020-03-13 17:12:00  25250.0  25250.00   

        LowPx   closePx  Volume Frequency              CandleDate  ...  \
0    22044.95  22055.65  250800

In [20]:
dfTZ.head


<bound method NDFrame.head of     InstrumentKey   MainDate MZ_GroupCode     TimeAtEventOpen  \
0       BANKNIFTY 2020-03-13           T0 2020-03-13 00:00:00   
1       BANKNIFTY 2020-03-13            A 2020-03-13 09:15:00   
2       BANKNIFTY 2020-03-13            A 2020-03-13 09:18:00   
3       BANKNIFTY 2020-03-13            A 2020-03-13 09:21:00   
4       BANKNIFTY 2020-03-13            C 2020-03-13 10:21:00   
..            ...        ...          ...                 ...   
101     BANKNIFTY 2020-03-13            L 2020-03-13 15:12:00   
102     BANKNIFTY 2020-03-13            L 2020-03-13 15:15:00   
103     BANKNIFTY 2020-03-13            M 2020-03-13 15:18:00   
104     BANKNIFTY 2020-03-13            M 2020-03-13 15:21:00   
105     BANKNIFTY 2020-03-13            M 2020-03-13 15:24:00   

    candle_PMA_Trend candle_PricePMA_Trend candle_Final_Trend  \
0           Waitmain              Waitmain               main   
1     StrongBearmain        StrongBearmain               ma

In [21]:
dfObsv.head

<bound method NDFrame.head of     Market InstrumentKey         Instrument  TradeDate MZ_GroupCode  \
0      NSE     BANKNIFTY  BANKNIFTY20MARFUT 2020-03-13            A   
1      NSE     BANKNIFTY  BANKNIFTY20MARFUT 2020-03-13            A   
2      NSE     BANKNIFTY  BANKNIFTY20MARFUT 2020-03-13            A   
3      NSE     BANKNIFTY  BANKNIFTY20MARFUT 2020-03-13            C   
4      NSE     BANKNIFTY  BANKNIFTY20MARFUT 2020-03-13            C   
..     ...           ...                ...        ...          ...   
100    NSE     BANKNIFTY  BANKNIFTY20MARFUT 2020-03-13            L   
101    NSE     BANKNIFTY  BANKNIFTY20MARFUT 2020-03-13            L   
102    NSE     BANKNIFTY  BANKNIFTY20MARFUT 2020-03-13            M   
103    NSE     BANKNIFTY  BANKNIFTY20MARFUT 2020-03-13            M   
104    NSE     BANKNIFTY  BANKNIFTY20MARFUT 2020-03-13            M   

        TimeAtEventOpen    TimeAtEventClose EventType EventDetail  \
0   2020-03-13 09:15:00 2020-03-13 09:15:00     

In [22]:
dfBehave.head

<bound method NDFrame.head of     InstrumentKey   MainDate           TradeDate       TradeTimeOnly  \
0       BANKNIFTY 2020-03-13 2020-03-13 09:15:00 1900-01-01 09:15:00   
1       BANKNIFTY 2020-03-13 2020-03-13 09:15:00 1900-01-01 09:15:00   
2       BANKNIFTY 2020-03-13 2020-03-13 09:15:00 1900-01-01 09:15:00   
3       BANKNIFTY 2020-03-13 2020-03-13 09:15:00 1900-01-01 09:15:00   
4       BANKNIFTY 2020-03-13 2020-03-13 09:15:00 1900-01-01 09:15:00   
..            ...        ...                 ...                 ...   
268     BANKNIFTY 2020-03-13 2020-03-13 15:21:00 1900-01-01 15:21:00   
269     BANKNIFTY 2020-03-13 2020-03-13 15:21:00 1900-01-01 15:21:00   
270     BANKNIFTY 2020-03-13 2020-03-13 15:21:00 1900-01-01 15:21:00   
271     BANKNIFTY 2020-03-13 2020-03-13 15:21:00 1900-01-01 15:21:00   
272     BANKNIFTY 2020-03-13 2020-03-13 15:24:00 1900-01-01 15:24:00   

    MZ_GroupCode             MainCode  MainCodeKey                LastStatus  \
0              A         

In [23]:
dfFpMap.head

<bound method NDFrame.head of    InstrumentKey   MainDate  ZoneDescription  ZoneColor  \
0      BANKNIFTY 2020-03-13    Fib138.2-100:       Pink   
1      BANKNIFTY 2020-03-13  Fib161.8-132.8:    HotPink   
2      BANKNIFTY 2020-03-13  Fib261.8-161.8:   DeepPink   
3      BANKNIFTY 2020-03-13    Fib61.8-38.2:       Blue   
4      BANKNIFTY 2020-03-13           H3-L3:     Yellow   
5      BANKNIFTY 2020-03-13           H4-H3:      Green   
6      BANKNIFTY 2020-03-13      H5-FREERUN:   Platinum   
7      BANKNIFTY 2020-03-13           H5-H4:     Golden   
8      BANKNIFTY 2020-03-13           L3-L4:        Red   
9      BANKNIFTY 2020-03-13           L4-L5:       Riot   
10     BANKNIFTY 2020-03-13     L5-FREEFALL:  BlackHole   
11     BANKNIFTY 2020-03-13         Prev-HL:       Gray   
12     BANKNIFTY 2020-03-13           R1-S1:     Yellow   
13     BANKNIFTY 2020-03-13           R2-R1:      Green   
14     BANKNIFTY 2020-03-13           R3-R2:     Golden   
15     BANKNIFTY 2020-03-1

In [35]:
print(dfTZ.columns)
for index,row in dfTZ.iterrows():
    print(row['zoneInfoColor'],row['zoneValueUpNet'],row['zoneValueDownNet'])
    



print('OK')

Index(['InstrumentKey', 'MainDate', 'MZ_GroupCode', 'TimeAtEventOpen',
       'candle_PMA_Trend', 'candle_PricePMA_Trend', 'candle_Final_Trend',
       'candle15_PMA_Trend', 'candle15_PricePMA_Trend', 'candle15_Final_Trend',
       ...
       'liveLast60MinuteCandlePMA200', 'liveYesterdayDayCandlePMA20',
       'liveYesterdayDayCandlePMA50', 'liveYesterdayDayCandlePMA200',
       'lastTMinus1120MinuteAnchor_OpenPx',
       'lastTMinus1120MinuteAnchor_HighPx', 'lastTMinus1120MinuteAnchor_LowPx',
       'lastTMinus1120MinuteAnchor_ClosePx', 'label1', 'label2'],
      dtype='object', length=208)
 0 0
_Red_Riot 0 -5
_Riot 0 -4
_BlackHole_Riot 0 -7
_Red_Yellow -1 -1
_Red_Riot 0 -4
_Red_Riot 0 -3
_Red_Yellow -2 1
_Gray_Orange_Yellow -4 4
_Gray_Orange_Yellow -4 4
_Gray_Orange_Yellow -4 4
_Gray_Orange_Yellow -4 4
_Gray_Orange_Yellow -4 4
_Blue_Gray_Orange_Yellow -5 5
_Blue_Gray_Green_Orange_Yellow -4 5
_Blue_Gray_Orange_Yellow -5 5
_Gray_Orange_Yellow -4 4
_Gray_Orange_Yellow -4 4
_Gray_Orange